# DI Pre Processing
This notebook extracts metadata from video files 

<br>

![UofC logo](./pictures/uofc_logo-black.jpg)

In [19]:
#import libraries
import os
import cv2
import json
import pandas as pd

In [20]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [21]:
# patient alias id
ALIAS_ID = {
  "Rachael B" : "1",
  "Sara" : "2",
  "Kate" : "3",
  "Nick" : "4",
  "Daniel" : "5",
  "Illia" : "6",
  "Melissa" : "7",
  "Karlii" : "8",
  "Eddy" : "9",
  "John McNeil" : "10",
  "Farooq" : "11",
  "Krystyn" : "12",
  "Jay" : "13",
  "Frank" : "14",
  "Arun" : "15",
  "Riyan" : "16",
  "Joseph" : "17",
  "John Brunton" : "18",
  "Illia2" : "19",
  "Frank2" : "20"
}

In [22]:
def get_video_properties(mp4_file: str) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        old_fps = video.get(cv2.CAP_PROP_FPS)
        number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
        length = number_of_frames / old_fps
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video.release()
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    
    return video_properties

In [23]:
def get_alias(patient_info:dict, camera:str) -> str:
    if camera == "rgb":
        return ALIAS_ID.get(patient_info["ID"], "?")
    
    if camera == "thermal":
        return patient_info["ID"]

In [24]:
def add_video_properties(metadata:dict, camera:str) -> dict:
    new_patient_info = {}
    
    for patient_id, patient_info in metadata.items():
        try:
            video_path = patient_info["local path"]
            frames, fps, length_in_seconds = get_video_properties(video_path)
            patient_info["frames"] = frames
            patient_info["old fps"] = fps
            patient_info["length"] = length_in_seconds
            patient_info["alias"] = get_alias(patient_info, camera)
            new_patient_info[patient_id] = patient_info
        except Exception as e:
            print(f"{type(e)} - {patient_info['ID']}")

    return new_patient_info 

In [25]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [26]:
def export_to_excel(patient_info:dict, filename: str) -> None:
    patient_df = pd.DataFrame.from_dict(patient_info).transpose()
    patient_df = patient_df.drop(columns=["local path"])
    new_df = patient_df.set_index(["ID", "distance", "breathing", "blanket", "filename"])
    new_df.to_excel(filename)
    return new_df

In [27]:
def export_to_json(filename: str, all_patient_info: dict) -> None:
    patient_json = json.dumps(all_patient_info, indent=2)

    with open(filename, "w") as json_data:
        json_data.write(patient_json)

## Get video metadata and export to xlsx file (stage 2.1)

In [28]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb.json")
metadata_thermal = load_json(json_dir, "thermal.json")

In [29]:
""" extract metadata from video (rgb) """
complete_metadata_rgb = add_video_properties(metadata_rgb, "rgb")

In [30]:
""" save to json (rgb) """
json_rgb_filename = repo_dir + "/records/JSON/rgb_complete.json"
export_to_json(json_rgb_filename, complete_metadata_rgb)

In [31]:
""" patient info to pandas DataFrame (rgb)"""
rgb_xlsx = repo_dir + "/records/xlsx/rgb.xlsx"
rgb_df = export_to_excel(complete_metadata_rgb, rgb_xlsx)

In [32]:
# """ extract metadata from video (thermal) """
# complete_metadata_thermal = add_video_properties(metadata_thermal, "thermal")

In [33]:
""" save to json (thermal) """
json_thermal_filename = repo_dir + "/records/JSON/thermal_complete.json"
export_to_json(json_thermal_filename, complete_metadata_thermal)

In [34]:
# """ patient info to pandas DataFrame (thermal)"""
# thermal_xlsx = repo_dir + "/records/xlsx/thermal.xlsx"
# thermal_df = export_to_excel(complete_metadata_thermal, thermal_xlsx)

In [35]:
""" read exported excel (rgb) """
new_rgb_df = pd.read_excel(rgb_xlsx)
new_rgb_df.describe()

,frames,length,old fps,alias
count,6.000000,6.000000,6.0,6.0
mean,1704.333333,28.405556,60.0,15.0
std,1263.451569,21.057526,0.0,0.0
min,143.000000,2.383333,60.0,15.0
25%,1137.250000,18.954167,60.0,15.0
50%,1464.000000,24.400000,60.0,15.0
75%,2057.000000,34.283333,60.0,15.0
max,3872.000000,64.533333,60.0,15.0


In [36]:
""" read exported excel (thermal) """
new_thermal_df = pd.read_excel(thermal_xlsx)
new_thermal_df.describe()

NameError: name 'thermal_xlsx' is not defined